# Restaurant Visitor Forecasting

### 1. Data Preprocessing

In [3]:
import pandas as pd

air_visit = pd.read_csv('data/air_visit_data.csv')
air_reserve = pd.read_csv('data/air_reserve.csv')
hpg_reserve = pd.read_csv('data/hpg_reserve.csv')
air_store_info = pd.read_csv('data/air_store_info.csv')
hpg_store_info = pd.read_csv('data/hpg_store_info.csv')
store_id_relation = pd.read_csv('data/store_id_relation.csv')
date_info = pd.read_csv('data/date_info.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')